In [1]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt         
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm
import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint
%matplotlib inline
from tensorflow.keras import models, layers

In [2]:
class_names = ['mountain', 'street', 'glacier', 'buildings', 'sea', 'forest']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

IMAGE_SIZE = (150, 150)



In [3]:
def load_data():
    """
        Load the data:
            - 14,034 images to train the network.
            - 3,000 images to evaluate how accurately the network learned to classify images.
    """
    
    datasets = ['../../../../archive/seg_train/seg_train', '../../../../archive/seg_test/seg_test']
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [4]:
(train_images, train_labels), (test_images, test_labels) = load_data()

  7%|▋         | 151/2191 [00:00<00:01, 1495.75it/s]

Loading ../../../../archive/seg_train/seg_train


 69%|██████▊   | 1558/2274 [00:00<00:00, 1646.71it/s]


KeyboardInterrupt: 

In [ ]:
train_images_scaled = train_images.reshape((-1,150,150,3)).astype('float32') / 255.0
test_images_scaled = test_images.reshape((-1,150,150,3)).astype('float32') / 255.0

In [ ]:
train_img_reshaped = train_images.reshape(train_images.shape[0],-1)
test_img_reshaped = test_images.reshape(test_images.shape[0],-1)

In [ ]:
dropout_model = models.Sequential()

In [ ]:
dropout_model.add(layers.Conv2D(64,(3,3),activation='relu',input_shape=(150, 150, 3)))
dropout_model.add(layers.MaxPooling2D(5,5))
dropout_model.add(layers.Dropout(0.3))
dropout_model.add(layers.Flatten())
dropout_model.add(layers.Dense(128,activation='relu'))
dropout_model.add(layers.Dense(6,activation='softmax'))

In [ ]:
dropout_model.summary()

In [ ]:
dropout_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [ ]:
#added checkpoints
checkpoint = tf.keras.callbacks.ModelCheckpoint("../../../../Models/baseline_dropout_model.h5", save_best_only=True)
earlystop = tf.keras.callbacks.EarlyStopping(
                                monitor='val_loss', # What to watch
                                min_delta=0.1, # How much change to get
                                patience=5 # No change after 5 epochs
)

In [ ]:
history = dropout_model.fit(train_images_scaled,train_labels,
                    batch_size=16,
                    epochs=5,
                    validation_data=(test_images_scaled,test_labels),
                    callbacks=[checkpoint,earlystop])

In [ ]:
metrics = ['acc','val_acc']
for metric in metrics:
    plt.plot(history.history[metric], label=metric)
plt.title('Dropout_Model-Accuracy')
plt.legend()
plt.tight_layout()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

In [ ]:
metrics = ['loss','val_loss']
for metric in metrics:
    plt.plot(history.history[metric], label=metric)
plt.title('Dropout_Model-Loss')

plt.legend()
plt.tight_layout()
plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
#try same model with resizing
dim = (75,75)

In [ ]:
def load_data():
    """
        Load the data:
            - 14,034 images to train the network.
            - 3,000 images to evaluate how accurately the network learned to classify images.
    """
    
    datasets = ['../../../../archive/seg_train/seg_train', '../../../../archive/seg_test/seg_test']
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, dim) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [ ]:
(train_images_resized, train_labels), (test_images_resized, test_labels) = load_data()

In [ ]:
def imshow(img):
    import cv2
    import IPython
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

In [ ]:
#before reshaping
imshow(train_images_resized[1])

In [ ]:
imshow(train_images[1])

In [ ]:
train_images_resized_scaled =train_images_resized.reshape((-1,75,75,3)).astype('float32') / 255.0
test_images_resized_scaled = test_images_resized.reshape((-1,75,75,3)).astype('float32') / 255.0

In [ ]:
dropout_model_resized_2 = models.Sequential()

In [ ]:
dropout_model_resized_2.add(layers.Conv2D(128,(3,3),activation='relu',input_shape=(75, 75, 3)))
dropout_model_resized_2.add(layers.MaxPooling2D(3,3))
dropout_model_resized_2.add(layers.Dropout(0.35))
dropout_model_resized_2.add(layers.Flatten())
dropout_model_resized_2.add(layers.Dense(128,activation='relu'))
dropout_model_resized_2.add(layers.Dense(6,activation='softmax'))

In [ ]:
dropout_model_resized_2.summary()

In [ ]:
dropout_model_resized_2.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [ ]:
#added checkpoints
checkpoint = tf.keras.callbacks.ModelCheckpoint("../../../../Models/resized_dropout_model_2.h5", save_best_only=True)
earlystop = tf.keras.callbacks.EarlyStopping(
                                monitor='val_loss', # What to watch
                                min_delta=0.1, # How much change to get
                                patience=5 # No change after 5 epochs
)

In [ ]:
history = dropout_model_resized_2.fit(train_images_resized_scaled,train_labels,
                    batch_size=16,
                    epochs=5,
                    validation_data=(test_images_resized_scaled,test_labels),
                    callbacks=[checkpoint,earlystop])

In [ ]:
metrics = ['acc','val_acc']
for metric in metrics:
    plt.plot(history.history[metric], label=metric)
plt.title('resized_Dropout_Model_2-Accuracy')
plt.legend()
plt.tight_layout()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

In [ ]:
metrics = ['loss','val_loss']
for metric in metrics:
    plt.plot(history.history[metric], label=metric)
plt.title('resized_Dropout_Model_2-Loss')

plt.legend()
plt.tight_layout()
plt.xlabel('Epochs')
plt.ylabel('Loss')

In [ ]:
def model_plots(history,name):
    plt.style.use('ggplot')
    
    fig1, ax1 = plt.subplots(figsize=(6,6))
    metrics = ['acc','val_acc']
    for metric in metrics:
        ax1 = plt.plot(history.history[metric], label=metric)
    ax1 = plt.title(f'{name}-Accuracy')

    ax1 = plt.legend()
    ax1 = plt.tight_layout()
    ax1 = plt.xlabel('Epochs')
    ax1 = plt.ylabel('Accuracy')
    fig1.savefig(f'Images/{name}-accuracy.jpg',bbox_inches='tight', dpi=150)
    
    fig2, ax2 = plt.subplots(figsize=(6,6))
    metrics = ['loss','val_loss']
    for metric in metrics:
        ax2 = plt.plot(history.history[metric], label=metric)
    ax2 = plt.title(f'{name}-Loss')

    ax2 = plt.legend()
    ax2 = plt.tight_layout()
    ax2 = plt.xlabel('Epochs')
    ax2 = plt.ylabel('Loss')
    fig2.savefig(f'Images/{name}-loss.jpg',bbox_inches='tight', dpi=150)

In [ ]:

def modeler(layers_list):
    model = models.Sequential()
    model.add(layers.Conv2D(128,(3,3),activation='relu',input_shape=(75, 75, 3)))

    for layer in layers_list:
        model.add(layer)
    
    model.add(layers.Flatten())
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dense(6,activation='softmax'))
    
    
 
    return model

In [ ]:
def model_trainer(model,name,train_img,train_labels,test_img,test_labels,batch_size=16,epochs=5):
    
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(f'../../../../Models/{name}_model.h5', save_best_only=True)
    earlystop = tf.keras.callbacks.EarlyStopping(
                                monitor='val_loss', # What to watch
                                min_delta=0.1, # How much change to get
                                patience=5 # No change after 5 epochs
    )
    history = model.fit(train_img,train_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(test_img,test_labels),
                    callbacks=[checkpoint,earlystop])
    
    return history

In [ ]:
layers_list = [layers.MaxPooling2D(3,3),
               layers.Dropout(0.35)]

model = modeler(layers_list)

In [ ]:
history = model_trainer(model,'test',train_images_resized_scaled,train_labels,test_images_resized_scaled,test_labels,epochs=5)

In [ ]:
model_plots(history,'test')